In [22]:
#!pip install langchain_community
#!pip install docarray
#!pip install langchain_chroma

In [30]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain_chroma import Chroma
from langchain.indexes import VectorstoreIndexCreator

from langchain_ibm import ChatWatsonx, WatsonxEmbeddings
from ibm_watsonx_ai import Credentials

In [53]:
parameters = {
    "temperature": 0.1,
    "max_tokens": 1000,
}

llm = ChatWatsonx(
    model_id="ibm/granite-3-8b-instruct",
    url="https://us-south.ml.cloud.ibm.com",
    apikey="0cZb46prORvZpwwPng_oLDJOxt5SmJwRqzw0gAfZKJ5_",
    project_id="2adbc31f-5dc4-4e42-b730-1e5e360727fd",
    params=parameters,
)

llm.invoke("what is the capital of Iran?")

AIMessage(content='The capital of Iran is Tehran.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 67, 'total_tokens': 78}, 'model_name': 'ibm/granite-3-8b-instruct', 'system_fingerprint': '', 'finish_reason': 'stop'}, id='chatcmpl-9a888f395c311c74c6e6a73cd9e8d6d5', usage_metadata={'input_tokens': 67, 'output_tokens': 11, 'total_tokens': 78})

In [ ]:
#https://github.com/IBM/watsonx-ai-samples/blob/master/cloud/notebooks/python_sdk/deployments/foundation_models/RAG/Use%20watsonx%2C%20Chroma%2C%20and%20LangChain%20to%20answer%20questions%20(RAG).ipynb

#https://www.ibm.com/architectures/papers/rag-cookbook/embedding

#https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models-embed.html?context=wx&pos=2&audience=wdp

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key="0cZb46prORvZpwwPng_oLDJOxt5SmJwRqzw0gAfZKJ5_"
)

embeddings = WatsonxEmbeddings(
    model_id="ibm/slate-30m-english-rtrvr",
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id="2adbc31f-5dc4-4e42-b730-1e5e360727fd"
    )

In [59]:
#embeddings.embed_query("The capital of Iran is Tehran.").shape

In [49]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file, encoding='latin1')

In [50]:
index = VectorstoreIndexCreator(vectorstore_cls=DocArrayInMemorySearch, embedding = embeddings).from_loaders([loader]) # vectorstore_cls=DocArrayInMemorySearch

In [51]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [45]:
query ="Please list all women leather shoes"

In [54]:
response = index.query(query, 
                       llm = llm)

display(Markdown(response))

| Name | Description |
| --- | --- |
| Tropical Breeze Shirt | A lightweight, breathable long-sleeve men's UPF shirt offering superior SunSmart protection from the sun's harmful rays. The wrinkle-resistant and moisture-wicking fabric keeps you cool and comfortable. It has a traditional fit and is UPF 50+ rated. |
| Sun Shield Shirt by | A high-performance sun shirt that blocks 98% of the sun's harmful UV rays. It has a slightly fitted design, falls at the hip, and is made of 78% nylon and 22% Lycra Xtra Life fiber. It is UPF 50+ rated and hand washable. |
| Men's Tropical Plaid Short-Sleeve Shirt | A lightest hot-weather shirt rated UPF 50+ for superior protection from the sun's UV rays. It has a traditional fit, is made of 100% polyester, and is wrinkle-resistant. It features front and back cape venting and two front bellows pockets. |
| Men's Plaid Tropic Shirt, Short-Sleeve | A Ultracomfortable sun protection shirt rated to UPF 50+. It is originally designed for fishing and offers UPF 50+ coverage. The high-performance fabric is wrinkle-free and quickly evaporates perspiration. It is made with 52% polyester and 48% nylon, and is machine washable and dryable. |

All shirts are UPF 50+ rated, providing the highest rated sun protection possible. They are designed to block 98% of the sun's harmful UV rays and are recommended by The Skin Cancer Foundation as an effective UV protectant. The fabrics are moisture-wicking, quick-drying, and wrinkle-resistant, ensuring comfort and convenience. The shirts also feature various design elements such as cape venting and pockets for added functionality.

## Step By Step

In [61]:
docs = loader.load()

db = DocArrayInMemorySearch.from_documents( docs, embeddings)

In [63]:
docs = db.similarity_search("pleae suggest a shirt with sunblocking")
list( docs )

[Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 709}, page_content=": 709\nname: Sunrise Tee\ndescription: Stay cool, comfortable and dry on the hottest days in our women's UV-protective button down shirt. The lightweight, high-performance fabric wicks away moisture and dries quickly.\n\nSize & Fit\nSlightly Fitted: Softly shapes the body. Falls at hip.\n\nWhy We Love It\nOur lightest hot-weather shirt lets you beat the heat. Originally designed for fishing, it's also a great choice for travel thanks to its wrinkle-free fabric and built-in sun protection with a rating of UPF 50+.\n\nFabric & Care\nLightweight performance synthetic wicks moisture, resists wrinkles and dries fast. Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester. Machine wash and dry.\n\nAdditional Features\nBuilt-in SunSmartâ\x84¢ UPF 50+ rated â\x80\x93 the highest rated sun protection possible. The high-performance fabric keeps you cool and comfortable by wicking perspiration away. 

In [81]:
retriever = db.as_retriever()

qdocs = "".join([docs[i].page_content for i in range(len(docs))])

response = llm.invoke(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.") 

display(Markdown(response.content))

| Name | Description | Size & Fit | Fabric & Care | Additional Features |
| --- | --- | --- | --- | --- | --- |
| Sunrise Tee | UV-protective button-down shirt for women, slightly fitted, wicks moisture, dries quickly, UPF 50+ | Slightly Fitted: Softly shapes the body. Falls at hip. | Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester. Machine wash and dry. | Built-in SunSmart UPF 50+ rated, wrinkle-free, fast-drying, smooth buttons, low-profile pockets, side shaping, front and back cape venting, two front pockets, tool tabs, eyewear loop. |
| Sun Shield Shirt | High-performance sun shirt for men, slightly fitted, wicks moisture, quick-drying, UPF 50+ | Slightly Fitted: Softly shapes the body. Falls at hip. | 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated. Handwash, line dry. | Wicks moisture, fits over swimsuits, abrasion-resistant, recommended by The Skin Cancer Foundation. |
| Tropical Breeze Shirt | Lightweight, breathable long-sleeve men's UPF shirt, traditional fit, wrinkle-resistant, moisture-wicking | Traditional Fit: Relaxed through the chest, sleeve, and waist. | Shell: 71% nylon, 29% polyester. Cape lining: 100% polyester. Polyester-mesh inserts. UPF 50+ rated. Machine wash and dry. | Wrinkle-resistant, front and back cape venting, two front bell pockets. |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightweight hot-weather shirt for men, traditional fit, UPF 50+, wrinkle-resistant | Traditional Fit: Relaxed through the chest, sleeve, and waist. | 100% polyester. UPF 50+ rated. Machine wash and dry. | Front and back cape venting, two front bellows pockets. |

All shirts listed above offer sun protection with UPF 50+ rating, ensuring 98% of harmful UV rays are blocked. They are designed for outdoor activities, particularly fishing, and are lightweight, breathable, and moisture-wicking to keep users cool and comfortable. The Sunrise Tee and Tropical Breeze Shirt are slightly fitted, while the Sun Shield Shirt and Men's Tropical Plaid Short-Sleeve Shirt have a traditional fit. The Sunrise Tee and Tropical Breeze Shirt are machine washable, while the Sun Shield Shirt requires hand washing and line drying. The Sun Shield Shirt and Men's Tropical Plaid Short-Sleeve Shirt are made of nylon and Lycra Xtra Life fiber or 100% polyester, respectively, and are abrasion-resistant for long-lasting use. The Sunrise Tee and Tropical Breeze Shirt have additional features like cape venting, pockets, and smooth buttons, while the Sun Shield Shirt is recommended by The Skin Cancer Foundation.

In [82]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

response = qa_stuff.invoke(query)

display(Markdown(response['result']))



> Entering new RetrievalQA chain...

> Finished chain.


| Name | Description | Sun Protection | Fabric | Care | Additional Features |
| --- | --- | --- | --- | --- | --- | --- |
| Sun Shield Shirt | High-performance sun shirt with UPF 50+ rating, blocking 98% of UV rays. Slightly fitted, made of 78% nylon and 22% Lycra Xtra Life fiber. Wicks moisture, abrasion resistant, handwash, line dry. | UPF 50+ | 78% nylon, 22% Lycra Xtra Life fiber | Handwash, line dry | Wicks moisture, abrasion resistant |
| Tropical Breeze Shirt | Lightweight, breathable long-sleeve men's UPF shirt with UPF 50+ rating. Traditional fit, made of 71% nylon and 29% polyester. Wrinkle-resistant, moisture-wicking, machine wash and dry. | UPF 50+ | 71% nylon, 29% polyester | Machine wash and dry | Wrinkle-resistant, moisture-wicking, front and back cape venting, two front pockets |
| Sunrise Tee | Women's UV-protective button-down shirt with UPF 50+ rating. Slightly fitted, made of 71% nylon and 29% polyester. Wicks moisture, resists wrinkles, machine wash and dry. | UPF 50+ | 71% nylon, 29% polyester | Machine wash and dry | Built-in SunSmart UPF 50+ rating, wrinkle-free, front and back cape venting, two front pockets, tool tabs, eyewear loop |
| Men's Tropical Plaid Short-Sleeve Shirt | Lightweight hot-weather shirt with UPF 50+ rating. Traditional fit, made of 100% polyester. Wrinkle-resistant, front and back cape venting, two front bellows pockets. | UPF 50+ | 100% polyester | Imported | Wrinkle-resistant, front and back cape venting, two front bellows pockets |

All shirts offer UPF 50+ sun protection, blocking 98% of the sun's harmful rays. They are designed for outdoor activities, providing comfort and protection from the sun. The fabrics are moisture-wicking, quick-drying, and wrinkle-resistant, ensuring long-lasting performance. Additional features include venting for cool breezes, pockets for convenience, and flattering fits for a comfortable wear.